# Summary Pipeline v4.0 - Complete Validation Notebook

This notebook allows you to explore and validate the summary pipeline results, covering:
1. **Basic data integrity** - row counts, duplicates, array lengths
2. **Gap scenarios** - accounts with missing months
3. **Backfill scenarios** - late-arriving data handling
4. **Rolling history correctness** - values shift properly

**Tables:**
- `default.default.accounts_all` - Source data
- `default.summary` - Summary with rolling history arrays
- `default.latest_summary` - Latest state per account

In [ ]:
# Initialize Spark Session
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime, timedelta

spark = SparkSession.builder \
    .appName("SummaryValidation") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

print("Spark session ready!")

---
## 1. Table Overview & Schema Validation

In [ ]:
# Table counts
print("=" * 70)
print("TABLE OVERVIEW")
print("=" * 70)

accounts_count = spark.table("default.default.accounts_all").count()
summary_count = spark.table("default.summary").count()
latest_count = spark.table("default.latest_summary").count()

print(f"Source (accounts_all):  {accounts_count:,} rows")
print(f"Summary:                {summary_count:,} rows")
print(f"Latest Summary:         {latest_count:,} rows")
print(f"\nUnique accounts in summary: {spark.table('default.summary').select('cons_acct_key').distinct().count():,}")
print(f"Unique months in summary:   {spark.table('default.summary').select('rpt_as_of_mo').distinct().count()}")

In [ ]:
# Full schema check - verify all expected columns exist
print("\nSUMMARY TABLE SCHEMA:")
print("=" * 70)

summary_cols = [f.name for f in spark.table("default.summary").schema.fields]

# Expected columns
expected_cols = [
    # Key columns
    'cons_acct_key', 'rpt_as_of_mo', 'base_ts',
    # Scalar columns
    'balance_am', 'days_past_due', 'actual_payment_am', 'credit_limit_am',
    'past_due_am', 'asset_class_cd', 'emi_amt',
    # History arrays
    'balance_am_history', 'days_past_due_history', 'actual_payment_am_history',
    'credit_limit_am_history', 'past_due_am_history', 'payment_rating_cd_history',
    'asset_class_cd_history',
    # Grid
    'payment_history_grid'
]

print(f"{'Column':<35} {'Present':<10} {'Type'}")
print("-" * 70)
for col in expected_cols:
    present = "YES" if col in summary_cols else "MISSING"
    col_type = str([f.dataType for f in spark.table("default.summary").schema.fields if f.name == col][0]) if col in summary_cols else "N/A"
    status = "" if present == "YES" else " <-- FIX NEEDED"
    print(f"{col:<35} {present:<10} {col_type}{status}")

In [ ]:
# Show sample data with key columns
print("\nSAMPLE DATA (key columns):")
spark.sql("""
    SELECT 
        cons_acct_key,
        rpt_as_of_mo,
        balance_am,
        days_past_due,
        actual_payment_am,
        asset_class_cd,
        SUBSTRING(payment_history_grid, 1, 12) as grid_first12
    FROM default.summary
    LIMIT 5
""").show(truncate=False)

---
## 2. Gap Scenario Validation

Test accounts that have **missing months** (gaps) in their data.

In [ ]:
# Find accounts with gaps in source data
print("=" * 70)
print("GAP SCENARIO ANALYSIS")
print("=" * 70)

# Get accounts with fewer months than expected
gap_accounts = spark.sql("""
    WITH all_months AS (
        SELECT DISTINCT rpt_as_of_mo FROM default.default.accounts_all ORDER BY rpt_as_of_mo
    ),
    account_months AS (
        SELECT 
            cons_acct_key,
            COUNT(DISTINCT rpt_as_of_mo) as month_count,
            COLLECT_SET(rpt_as_of_mo) as months_present
        FROM default.default.accounts_all
        GROUP BY cons_acct_key
    )
    SELECT 
        cons_acct_key,
        month_count,
        (SELECT COUNT(*) FROM all_months) as total_months,
        (SELECT COUNT(*) FROM all_months) - month_count as missing_months
    FROM account_months
    WHERE month_count < (SELECT COUNT(*) FROM all_months)
    ORDER BY missing_months DESC
    LIMIT 10
""")

print(f"\nAccounts with gaps (missing months in source):")
gap_accounts.show()

In [ ]:
# Pick a gap account and analyze it
gap_account_row = gap_accounts.first()
if gap_account_row:
    gap_account = gap_account_row['cons_acct_key']
    print(f"\nAnalyzing gap account: {gap_account}")
    print("=" * 70)
    
    # Source data for this account
    print("\nSource data (accounts_all):")
    spark.sql(f"""
        SELECT rpt_as_of_mo, acct_bal_am as balance, days_past_due_ct_4in as dpd
        FROM default.default.accounts_all
        WHERE cons_acct_key = {gap_account}
        ORDER BY rpt_as_of_mo
    """).show()
    
    # Summary data for this account
    print("\nSummary data (with history arrays):")
    spark.sql(f"""
        SELECT 
            rpt_as_of_mo,
            balance_am,
            days_past_due,
            SLICE(balance_am_history, 1, 8) as bal_hist_first8,
            payment_history_grid
        FROM default.summary
        WHERE cons_acct_key = {gap_account}
        ORDER BY rpt_as_of_mo
    """).show(truncate=False)
else:
    print("No gap accounts found in source data.")

In [ ]:
# Verify gap handling - NULL values should appear in history for missing months
print("\nGAP HANDLING VERIFICATION:")
print("When a month is missing, the history array should have NULL at that position.")
print("=" * 70)

if gap_account_row:
    # Get the history for the latest month
    latest_hist = spark.sql(f"""
        SELECT 
            rpt_as_of_mo,
            balance_am_history,
            days_past_due_history
        FROM default.summary
        WHERE cons_acct_key = {gap_account}
        ORDER BY rpt_as_of_mo DESC
        LIMIT 1
    """).collect()[0]
    
    bal_hist = latest_hist['balance_am_history']
    dpd_hist = latest_hist['days_past_due_history']
    
    print(f"\nAccount {gap_account} - Latest month: {latest_hist['rpt_as_of_mo']}")
    print(f"\nBalance history (first 12 positions):")
    for i, val in enumerate(bal_hist[:12]):
        marker = "<-- NULL (gap)" if val is None else ""
        print(f"  [{i}] {val} {marker}")
    
    null_count = sum(1 for v in bal_hist if v is None)
    print(f"\nTotal NULLs in balance history: {null_count} out of {len(bal_hist)}")

---
## 3. Backfill Scenario Test

Simulate late-arriving data and test backfill processing.

In [ ]:
# Check if there's any backfill data (records with later base_ts for earlier months)
print("=" * 70)
print("BACKFILL SCENARIO ANALYSIS")
print("=" * 70)

backfill_candidates = spark.sql("""
    WITH ranked AS (
        SELECT 
            cons_acct_key,
            rpt_as_of_mo,
            base_ts,
            ROW_NUMBER() OVER (PARTITION BY cons_acct_key ORDER BY rpt_as_of_mo) as month_rank,
            ROW_NUMBER() OVER (PARTITION BY cons_acct_key ORDER BY base_ts) as ts_rank
        FROM default.default.accounts_all
    )
    SELECT cons_acct_key, COUNT(*) as out_of_order_count
    FROM ranked
    WHERE month_rank != ts_rank
    GROUP BY cons_acct_key
    ORDER BY out_of_order_count DESC
    LIMIT 10
""")

backfill_count = backfill_candidates.count()
print(f"\nAccounts with out-of-order timestamps (potential backfill): {backfill_count}")
if backfill_count > 0:
    backfill_candidates.show()

In [ ]:
# Create a simulated backfill scenario for testing
print("\nSIMULATED BACKFILL TEST")
print("=" * 70)
print("We'll insert a new record for an old month and run backfill processing.")
print("\nCurrent state before backfill:")

# Pick an account to test backfill
test_account = spark.sql("""
    SELECT cons_acct_key 
    FROM default.summary 
    GROUP BY cons_acct_key 
    HAVING COUNT(*) >= 3
    LIMIT 1
""").collect()[0][0]

print(f"\nTest account: {test_account}")
print("\nCurrent summary history:")
spark.sql(f"""
    SELECT 
        rpt_as_of_mo,
        balance_am,
        SLICE(balance_am_history, 1, 6) as bal_hist
    FROM default.summary
    WHERE cons_acct_key = {test_account}
    ORDER BY rpt_as_of_mo
""").show(truncate=False)

In [ ]:
# Insert backfill test data
print("\nInserting backfill test record...")

# Get an existing record to use as template
template = spark.sql(f"""
    SELECT * FROM default.default.accounts_all
    WHERE cons_acct_key = {test_account}
    ORDER BY rpt_as_of_mo
    LIMIT 1
""").collect()[0]

# Create backfill record for month 2024-02 with updated balance
backfill_month = "2024-02"
new_balance = 999999  # Distinctive value to identify

print(f"Creating backfill record for month {backfill_month} with balance {new_balance}")
print("(This would be inserted with a newer base_ts to simulate late arrival)")

# Show what the backfill record would look like
print(f"""
Backfill record:
  cons_acct_key: {test_account}
  rpt_as_of_mo:  {backfill_month}
  balance:       {new_balance}
  base_ts:       {datetime.now()} (newer than existing)
""")

In [ ]:
# Explain backfill logic
print("\nBACKFILL PROCESSING LOGIC:")
print("=" * 70)
print("""
When backfill mode runs, it:

1. CASE I (New Account):
   - Account doesn't exist in summary
   - Creates new row with [value, NULL, NULL, ...]

2. CASE II (Forward Entry):
   - New month > max existing month
   - Shifts history: [new_value, gap_nulls, old_history...]

3. CASE III (Backfill - Historical Update):
   - New month <= max existing month AND new.base_ts > existing.base_ts
   - Rebuilds entire history for affected account
   - Updates all future months' history arrays

To test backfill manually:
  1. Insert a record with an old month but new timestamp
  2. Run: spark-submit summary_pipeline.py --config summary_config.json --mode backfill
  3. Verify the history arrays are rebuilt correctly
""")

---
## 4. Rolling History Validation

In [ ]:
# Verify history shift is correct
print("=" * 70)
print("ROLLING HISTORY SHIFT VALIDATION")
print("=" * 70)

# Pick an account with full history
full_account = spark.sql("""
    SELECT cons_acct_key 
    FROM default.summary 
    GROUP BY cons_acct_key 
    HAVING COUNT(*) = (SELECT COUNT(DISTINCT rpt_as_of_mo) FROM default.summary)
    LIMIT 1
""").collect()

if full_account:
    full_account = full_account[0][0]
    print(f"\nAccount with all months: {full_account}")
    
    # Get source values
    source_df = spark.sql(f"""
        SELECT rpt_as_of_mo, acct_bal_am as balance
        FROM default.default.accounts_all
        WHERE cons_acct_key = {full_account}
        ORDER BY rpt_as_of_mo
    """)
    
    print("\nSource values:")
    source_df.show()
    
    # Get summary history
    print("\nHistory arrays over time (showing how values shift):")
    spark.sql(f"""
        SELECT 
            rpt_as_of_mo,
            balance_am_history[0] as h0_current,
            balance_am_history[1] as h1_prev1,
            balance_am_history[2] as h2_prev2,
            balance_am_history[3] as h3_prev3,
            balance_am_history[4] as h4_prev4,
            balance_am_history[5] as h5_prev5
        FROM default.summary
        WHERE cons_acct_key = {full_account}
        ORDER BY rpt_as_of_mo
    """).show()
else:
    print("No account found with all months. Using first available.")

In [ ]:
# Verify source matches history[0]
print("\nVALIDATION: Source value = history[0]")
print("=" * 70)

mismatch_check = spark.sql("""
    SELECT 
        a.cons_acct_key,
        a.rpt_as_of_mo,
        a.acct_bal_am as source_balance,
        s.balance_am_history[0] as history_0,
        CASE WHEN a.acct_bal_am = s.balance_am_history[0] THEN 'MATCH' ELSE 'MISMATCH' END as status
    FROM default.default.accounts_all a
    JOIN default.summary s 
        ON a.cons_acct_key = s.cons_acct_key 
        AND a.rpt_as_of_mo = s.rpt_as_of_mo
    WHERE a.acct_bal_am != s.balance_am_history[0]
    LIMIT 10
""")

mismatch_count = mismatch_check.count()
if mismatch_count == 0:
    print("PASSED: All source values match history[0] values")
else:
    print(f"FAILED: {mismatch_count} mismatches found")
    mismatch_check.show()

In [ ]:
# Verify history[1] = previous month's history[0]
print("\nVALIDATION: history[1] should equal previous month's history[0]")
print("=" * 70)

shift_check = spark.sql("""
    WITH ordered AS (
        SELECT 
            cons_acct_key,
            rpt_as_of_mo,
            balance_am_history[0] as current_h0,
            balance_am_history[1] as current_h1,
            LAG(balance_am_history[0]) OVER (PARTITION BY cons_acct_key ORDER BY rpt_as_of_mo) as prev_h0
        FROM default.summary
    )
    SELECT *
    FROM ordered
    WHERE prev_h0 IS NOT NULL  -- Skip first month
      AND current_h1 != prev_h0
      AND current_h1 IS NOT NULL  -- Allow for gaps
    LIMIT 10
""")

shift_errors = shift_check.count()
if shift_errors == 0:
    print("PASSED: History shift is correct (h[1] = prev month's h[0])")
else:
    print(f"FAILED: {shift_errors} shift errors found")
    shift_check.show()

---
## 5. Array Length & Data Integrity Checks

In [ ]:
# Check all array lengths
print("=" * 70)
print("ARRAY LENGTH VALIDATION")
print("=" * 70)

array_columns = [
    'balance_am_history',
    'days_past_due_history', 
    'actual_payment_am_history',
    'credit_limit_am_history',
    'past_due_am_history',
    'payment_rating_cd_history',
    'asset_class_cd_history'
]

for col in array_columns:
    result = spark.sql(f"""
        SELECT 
            '{col}' as column_name,
            SUM(CASE WHEN SIZE({col}) = 36 THEN 1 ELSE 0 END) as correct,
            SUM(CASE WHEN SIZE({col}) != 36 THEN 1 ELSE 0 END) as incorrect
        FROM default.summary
    """).collect()[0]
    
    status = "PASS" if result['incorrect'] == 0 else "FAIL"
    print(f"{col:<35} {status} (correct: {result['correct']}, incorrect: {result['incorrect']})")

In [ ]:
# Check for duplicates
print("\nDUPLICATE CHECK")
print("=" * 70)

duplicates = spark.sql("""
    SELECT cons_acct_key, rpt_as_of_mo, COUNT(*) as cnt
    FROM default.summary
    GROUP BY cons_acct_key, rpt_as_of_mo
    HAVING COUNT(*) > 1
""").count()

if duplicates == 0:
    print("PASS: No duplicate (account, month) combinations")
else:
    print(f"FAIL: {duplicates} duplicate combinations found")

In [ ]:
# Check payment history grid
print("\nPAYMENT HISTORY GRID CHECK")
print("=" * 70)

grid_check = spark.sql("""
    SELECT 
        LENGTH(payment_history_grid) as grid_length,
        COUNT(*) as count
    FROM default.summary
    GROUP BY LENGTH(payment_history_grid)
""")

grid_check.show()

# Validate grid content
print("\nGrid character distribution (first position):")
spark.sql("""
    SELECT 
        SUBSTRING(payment_history_grid, 1, 1) as first_char,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as pct
    FROM default.summary
    GROUP BY 1
    ORDER BY count DESC
""").show()

---
## 6. DPD and Payment Rating Analysis

In [ ]:
# DPD distribution
print("=" * 70)
print("DAYS PAST DUE DISTRIBUTION")
print("=" * 70)

spark.sql("""
    SELECT 
        CASE 
            WHEN days_past_due IS NULL THEN 'NULL'
            WHEN days_past_due = 0 THEN '0 (Current)'
            WHEN days_past_due BETWEEN 1 AND 29 THEN '1-29 DPD'
            WHEN days_past_due BETWEEN 30 AND 59 THEN '30-59 DPD'
            WHEN days_past_due BETWEEN 60 AND 89 THEN '60-89 DPD'
            WHEN days_past_due BETWEEN 90 AND 119 THEN '90-119 DPD'
            WHEN days_past_due BETWEEN 120 AND 179 THEN '120-179 DPD'
            WHEN days_past_due >= 180 THEN '180+ DPD'
            ELSE 'Other'
        END as dpd_bucket,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as pct
    FROM default.summary
    GROUP BY 1
    ORDER BY 
        CASE 
            WHEN days_past_due IS NULL THEN -1
            WHEN days_past_due = 0 THEN 0
            WHEN days_past_due BETWEEN 1 AND 29 THEN 1
            WHEN days_past_due BETWEEN 30 AND 59 THEN 2
            WHEN days_past_due BETWEEN 60 AND 89 THEN 3
            WHEN days_past_due BETWEEN 90 AND 119 THEN 4
            WHEN days_past_due BETWEEN 120 AND 179 THEN 5
            ELSE 6
        END
""").show(truncate=False)

In [ ]:
# Verify DPD matches source
print("\nDPD SOURCE MATCH VALIDATION")
print("=" * 70)

dpd_mismatch = spark.sql("""
    SELECT COUNT(*) as mismatch_count
    FROM default.default.accounts_all a
    JOIN default.summary s 
        ON a.cons_acct_key = s.cons_acct_key 
        AND a.rpt_as_of_mo = s.rpt_as_of_mo
    WHERE a.days_past_due_ct_4in != s.days_past_due
""").collect()[0]['mismatch_count']

if dpd_mismatch == 0:
    print("PASS: All DPD values match between source and summary")
else:
    print(f"FAIL: {dpd_mismatch} DPD mismatches found")

---
## 7. Interactive Account Explorer

In [ ]:
# Change this to explore any account
EXPLORE_ACCOUNT = 100

print("=" * 70)
print(f"ACCOUNT {EXPLORE_ACCOUNT} DEEP DIVE")
print("=" * 70)

# Check if account exists
exists = spark.sql(f"SELECT COUNT(*) as cnt FROM default.summary WHERE cons_acct_key = {EXPLORE_ACCOUNT}").collect()[0]['cnt']

if exists > 0:
    print(f"\nAccount found with {exists} month(s) of data")
    
    # Source data
    print("\n--- Source Data ---")
    spark.sql(f"""
        SELECT 
            rpt_as_of_mo, 
            acct_bal_am as balance,
            days_past_due_ct_4in as dpd,
            actual_pymt_am as payment,
            asset_class_cd_4in as asset_class
        FROM default.default.accounts_all 
        WHERE cons_acct_key = {EXPLORE_ACCOUNT}
        ORDER BY rpt_as_of_mo
    """).show(truncate=False)
    
    # Summary data
    print("\n--- Summary Data ---")
    spark.sql(f"""
        SELECT 
            rpt_as_of_mo,
            balance_am,
            days_past_due,
            actual_payment_am,
            asset_class_cd,
            payment_history_grid
        FROM default.summary
        WHERE cons_acct_key = {EXPLORE_ACCOUNT}
        ORDER BY rpt_as_of_mo
    """).show(truncate=False)
    
    # Latest history arrays
    print("\n--- History Arrays (latest month, first 8 positions) ---")
    latest = spark.sql(f"""
        SELECT 
            rpt_as_of_mo,
            SLICE(balance_am_history, 1, 8) as balance_hist,
            SLICE(days_past_due_history, 1, 8) as dpd_hist,
            SLICE(payment_rating_cd_history, 1, 8) as rating_hist
        FROM default.summary
        WHERE cons_acct_key = {EXPLORE_ACCOUNT}
        ORDER BY rpt_as_of_mo DESC
        LIMIT 1
    """).show(truncate=False)
else:
    print(f"Account {EXPLORE_ACCOUNT} not found. Try another account ID.")

---
## 8. Summary Report

In [ ]:
# Final validation summary
print("=" * 70)
print("VALIDATION SUMMARY REPORT")
print("=" * 70)

checks = []

# Check 1: Row counts match
source_count = spark.table("default.default.accounts_all").count()
summary_count = spark.table("default.summary").count()
checks.append(("Row counts (source = summary)", source_count == summary_count, f"{source_count} vs {summary_count}"))

# Check 2: No duplicates
dup_count = spark.sql("""
    SELECT COUNT(*) FROM (
        SELECT cons_acct_key, rpt_as_of_mo FROM default.summary
        GROUP BY 1, 2 HAVING COUNT(*) > 1
    )
""").collect()[0][0]
checks.append(("No duplicates", dup_count == 0, f"{dup_count} duplicates"))

# Check 3: Array lengths
bad_arrays = spark.sql("""
    SELECT COUNT(*) FROM default.summary
    WHERE SIZE(balance_am_history) != 36 OR SIZE(days_past_due_history) != 36
""").collect()[0][0]
checks.append(("Array lengths = 36", bad_arrays == 0, f"{bad_arrays} bad arrays"))

# Check 4: Grid length
bad_grids = spark.sql("""
    SELECT COUNT(*) FROM default.summary WHERE LENGTH(payment_history_grid) != 36
""").collect()[0][0]
checks.append(("Grid length = 36", bad_grids == 0, f"{bad_grids} bad grids"))

# Check 5: Balance match
bal_mismatch = spark.sql("""
    SELECT COUNT(*) FROM default.default.accounts_all a
    JOIN default.summary s ON a.cons_acct_key = s.cons_acct_key AND a.rpt_as_of_mo = s.rpt_as_of_mo
    WHERE a.acct_bal_am != s.balance_am_history[0]
""").collect()[0][0]
checks.append(("Balance source = history[0]", bal_mismatch == 0, f"{bal_mismatch} mismatches"))

# Check 6: DPD match
dpd_mismatch = spark.sql("""
    SELECT COUNT(*) FROM default.default.accounts_all a
    JOIN default.summary s ON a.cons_acct_key = s.cons_acct_key AND a.rpt_as_of_mo = s.rpt_as_of_mo
    WHERE a.days_past_due_ct_4in != s.days_past_due
""").collect()[0][0]
checks.append(("DPD source = summary", dpd_mismatch == 0, f"{dpd_mismatch} mismatches"))

# Print results
print(f"\n{'Check':<40} {'Status':<10} {'Details'}")
print("-" * 70)

passed = 0
for check_name, check_passed, details in checks:
    status = "PASS" if check_passed else "FAIL"
    passed += 1 if check_passed else 0
    print(f"{check_name:<40} {status:<10} {details}")

print("-" * 70)
print(f"\nTotal: {passed}/{len(checks)} checks passed")

if passed == len(checks):
    print("\n ALL VALIDATIONS PASSED! ")
else:
    print("\n SOME VALIDATIONS FAILED - Review above ")

In [ ]:
# Custom query cell
print("\nRun your own queries below:")

# Example: Find accounts with high DPD
spark.sql("""
    SELECT 
        cons_acct_key,
        rpt_as_of_mo,
        days_past_due,
        balance_am,
        payment_history_grid
    FROM default.summary
    WHERE days_past_due > 90
    ORDER BY days_past_due DESC
    LIMIT 5
""").show(truncate=False)